#** Machine learning IB-2024** #** Home task 1.** #Regression, KNN, LinearRegression.

In this homework we will build models to predict the price of an apartment in Russia. below is a description of some columns of the data set. date - date of publication price - price in rubles level- floor, on which is the apartment levels - number of floors in the apartment rooms - number of rooms in the apartment. If value -1, then the apartment is considered apartments. area - area of the apartment. kitchen_area - area of the kitchen. geo_lat - Latitude geo_lon - Longitude building_type - building material. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

Part 0 – Start of work

To start working with the data, we import the libraries that will be needed in this task.

In [43]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

**Load** the folium library to display the data on the map by coordinates.

In [44]:
!pip install folium

We delete our data from the archive.

In [45]:
!unzip ...

unzip:  cannot find or open ..., ....zip or ....ZIP.


Download the data from the CSV file into the data frame.

In [46]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/input_data.csv'
df = pd.read_csv(path, sep = ';')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Show the coordinates of our buildings on the map.

In [47]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# List of points with width and length
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Adding points to the map
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

Part 1: Preparing data for processing by machine learning models.

**0.5 Ball**. The geography of our observations in the data set is extremely large. However, we know that the cost of apartments in Moscow and St. Petersburg is much higher than the average in Russia. Let's make signs that show whether the apartment is 20 kilometers from the centre of Moscow or whether the apartment is 20 kilometers from the centre of St. Petersburg. Create two signs is_Moscow and is_Saint_Peterburg. To find the distance by the coordinates use the function haversine_distance.

In [48]:
def haversine_distance(lat1, lon1, lat2, lon2):
  R = 6371
  dif_lat = (lat2 - lat1) / (2 * 180 / 3.14)
  dif_lon = (lon2 - lon1) / (2 * 180 / 3.14)
  return 2 * R * np.arcsin(((np.sin(dif_lat) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dif_lon)) ** 2)) ** 0.5)
lat_M = 55.558741
lon_M = 37.378847
lat_SP = 59.9386
lon_SP = 30.3141
df['is_Moscow'] = haversine_distance(lat_M, lon_M, df['geo_lat'], df['geo_lon']) < 20
df['is_Saint_Peterburg'] = haversine_distance(lat_SP, lon_SP, df['geo_lat'], df['geo_lon']) < 20

**0.5 Ball**. In our data set there are signs that we can theoretically use, such as postal_code, but we will do this within the framework of homework very-very long. Therefore it is proposed to remove unnecessary signs from the data frame. We will remove geo_lat, geo_lon, object_type, postal_code, street_id, id_region, house_id.

In [49]:
df.drop(['geo_lat',
         'geo_lon',
         'object_type',
         'postal_code',
         'street_id',
         'id_region',
         'house_id'], axis=1, inplace=True)

What columns are categorical? What numbers are? Categorical: (Your answer) Numbers: (Your answer) Let us encode categorical characters using OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [50]:
from sklearn.preprocessing import OneHotEncoder
a = OneHotEncoder(sparse_output = False)
df_enc = pd.DataFrame(a.fit_transform(df[['building_type', 'is_Moscow', 'is_Saint_Peterburg']]))
df = df.join(df_enc)
df.drop(['building_type', 'is_Moscow', 'is_Saint_Peterburg'], axis = 1, inplace = True)
df

,date,price,level,levels,rooms,area,kitchen_area,0,1,2,3,4,5,6,7,8,9,10,11
0,2021-01-01,2451300,15,31,1,30.3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,2021-01-01,10700000,4,13,3,85.0,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2021-01-01,2500000,2,3,1,30.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9499458,2021-11-01,7300000,7,9,1,36.0,7.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9499459,2021-11-01,11992854,12,23,2,67.0,-100.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9499460,2021-11-01,5600000,2,7,-1,14.0,-100.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
9499461,2021-11-01,9700000,2,7,-1,25.0,-100.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


**0.5 Ball**. We work with numerical characters: 1. Add two characters to your dataset: the number of days from the day of the first observation (the difference between the dates of the ads). Maybe for the price forecast the floor is not as important as the ratio of the floor to the number of floors in the house, add this character. After adding a new character the date column can be removed. 2. The numerical characters may have different orders. Let us compare the numerical characters using StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

In [51]:
from sklearn.preprocessing import StandardScaler
date_min = pd.to_datetime(df['date'].min())
unique_level = df['level'].unique()
unique_levels = df['levels'].unique()
print(unique_levels) # Verification
df['date'] = pd.to_datetime(df['date'])
df['date_dif'] = (df['date'] - date_min).dt.days
df['level_ratio'] = np.where(
    df['level'] == 0,
    df['levels'] + 1,
    df['levels'] / df['level']
)
df.drop('date', axis = 1, inplace = True)
colm = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'level_ratio', 'date_dif']
scaler = StandardScaler()
df[colm] = scaler.fit_transform(df[colm])
df

[31  5 13  3  4  2 18 19  9 17 20  7  6 22 29 14  8 10 16  1 25 11 12 21
 24 33 42 15 32 26 23 27 47 34 39 28 41 38 30 36 35 40 44 37 46 43 45 48
 49 50  0]


,price,level,levels,rooms,area,kitchen_area,0,1,2,3,4,5,6,7,8,9,10,11,date_dif,level_ratio
0,-0.019819,1.625614,2.672767,-0.627615,-0.840927,0.101469,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.872269,-0.253760
1,-0.024467,-0.267504,-0.932213,-0.627615,-0.741997,0.282028,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.872269,-0.642925
2,0.018471,-0.456816,0.177012,1.106700,1.163309,0.462587,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.872269,0.177971
3,-0.016808,-0.646128,-0.932213,1.106700,1.053388,0.372307,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.872269,-0.399697
4,-0.019593,-0.835440,-1.209520,-0.627615,-0.851919,0.372307,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.872269,-0.460504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9499458,0.002689,0.111119,-0.377601,-0.627615,-0.632076,0.327168,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.705750,-0.538685
9499459,0.024473,1.057678,1.563543,0.239542,0.503780,-2.907851,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.705750,-0.308486
9499460,-0.005203,-0.835440,-0.654907,-2.361929,-1.438167,-2.907851,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.705750,0.269181
9499461,0.013829,-0.835440,-0.654907,-2.361929,-1.035122,-2.907851,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.705750,0.269181


**2 Ball**. Realize the KNNRegressor class, which should make regression by the method to the nearest neighbors.

In [52]:
from scipy.spatial.distance import euclidean
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric=euclidean):
      self.n_neighbors = n_neighbors
      self.metric = metric

    def fit(self, X, y):
      self.X = X
      self.y = y
      return self

    def predict(self, X_t):
      Y_pr = np.empty(X_t.shape[0])
      for index, entry in enumerate(X_t):
        distances = [self.metric(i, entry) for i in self.X]
        neighbors = np.argsort(distances)[: self.n_neighbors]
        neighbors_prediction = [self.y[j] for j in neighbors]
        Y_pr[index] = np.mean(neighbors_prediction)
      return Y_pr

**3 Balls**. Realize the LinearRegression class that supports gradient leaks SGD, Momentum, AdaGrad. Use gradient to optimize the MSE loss function.

In [55]:
import numpy as np

class LinearRegression:
    def __init__(self, step=0.01, optimizer='SGD', eps=1e-8, momentum=0.9,
                 epochs=1000, tolerance=1e-6):
        self.step = step
        self.optimizer = optimizer
        self.eps = eps
        self.momentum = momentum
        self.epochs = epochs
        self.tolerance = tolerance

        self.coef_ = None
        self.offset_ = 0.0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.coef_ = np.zeros(n_features)
        self.offset_ = 0.0
        vel_w = np.zeros_like(self.coef_)
        vel_b = 0.0
        acc_w = np.zeros_like(self.coef_)
        acc_b = 0.0

        prev_loss = float('inf')

        for epoch in range(self.epochs):
            prediction = X @ self.coef_ + self.offset_
            diff = prediction - y
            grad_coef = (2 / n_samples) * (X.T @ diff)
            grad_bias = (2 / n_samples) * np.sum(diff)
            if self.optimizer == 'SGD':
                self.coef_ -= self.step * grad_coef
                self.offset_ -= self.step * grad_bias

            elif self.optimizer == 'Momentum':
                vel_w = self.momentum * vel_w + self.step * grad_coef
                vel_b = self.momentum * vel_b + self.step * grad_bias
                self.coef_ -= vel_w
                self.offset_ -= vel_b

            elif self.optimizer == 'AdaGrad':
                acc_w += grad_coef ** 2
                acc_b += grad_bias ** 2
                self.coef_ -= (self.step / (np.sqrt(acc_w) + self.eps)) * grad_coef
                self.offset_ -= (self.step / (np.sqrt(acc_b) + self.eps)) * grad_bias

            loss = np.mean(diff ** 2)
            if abs(prev_loss - loss) < self.tolerance:
                break
            prev_loss = loss

        return self

    def predict(self, X):
        return X @ self.coef_ + self.offset_


Part 2: Experiments with machine learning models.

**3 Balls**. Do experiments with the machine learning methods you have written. Separate the training and test samples at 0.8 and 0.2 respectively. Measure the MSE, MAE, RMSE error. Use the KNNRegressor and LinearRegression methods from the sklearn library, compare the quality of your solutions and library.